In [163]:
import math

In [164]:
ship_grid = [[0, 1, 0], [1,1,1], [1,0,1]]
torpedo_grid = [[0,1,0], [0,1,0],[0,0,0]]

In [165]:
name_obj1 = "ship"
name_obj2 = "torpedo"

In [ ]:
def place_object_on_grid(grid, obj, x, y):
    obj_height = len(obj)
    obj_width = len(obj[0]) 
    for i in range(obj_height):
        for j in range(obj_width):
            grid[x + i][y + j] += obj[i][j]

    return grid

In [167]:
def check_collision(object1, object2, grid_size_x, grid_size_y, pos_x, pos_y):
    grid = [[0 for _ in range(-2*grid_size_x,2*grid_size_x + len(object2[0]))] for _ in range(-2*grid_size_y, 2*grid_size_y + len(object2))]
    grid = place_object_on_grid(place_object_on_grid(grid, object1,2*grid_size_x, 2*grid_size_x)
                                , object2
                                , 2*grid_size_x - pos_x
                                , 2*grid_size_y + pos_y)


    #for i in range(len(grid)):
    #    print(grid[i])
    return any(cell == 2 for row in grid for cell in row)


In [168]:
def brezenham_algorithm(point1_x, point2_x, point1_y, point2_y):
    points = [(point1_x, point1_y), (point2_x, point2_y)]
    deltax = abs(point2_x - point1_x)
    deltay = abs(point2_y - point1_y)
    error = 0
    deltaerr = (deltay + 1)
    y = point1_y
    diry = point2_y - point1_y
    if diry > 0:
        diry = 1
    elif diry < 0:
        diry = -1
    for x in range(point1_x, point2_x+1):
        points.append((x, y))
        error += deltaerr
        if error >= (deltax + 1):
            y = y + diry
            error -= (deltax + 1)
    return points

In [169]:
def turning(object, angle): # список точек, собирающий в implementing наш объект
    size_x = len(object[0])
    size_y = len(object)
    points = []
    for column in range(size_x): # столбец известен (x)
        start = -1
        end = -1
        for row in range(size_y):
            if object[row][column] == 1:
                if start == -1:
                    start = row
                end = row
        if start != -1 and end != -1: # центр находится в середине куба
            center_x = size_x // 2
            center_y = size_y // 2
            new_start_x = (column-center_x)*math.cos(math.radians(angle)) - (start - center_y)*math.sin(math.radians(angle))
            new_start_y = (column-center_x)*math.sin(math.radians(angle)) + (start - center_y)*math.cos(math.radians(angle))
            new_end_x = (column-center_x)*math.cos(math.radians(angle)) - (end - center_y)*math.sin(math.radians(angle))
            new_end_y = (column-center_x)*math.sin(math.radians(angle)) + (end - center_y)*math.cos(math.radians(angle))

            new_start_x += center_x
            new_start_y += center_y
            new_end_x += center_x
            new_end_y += center_y

            points += brezenham_algorithm(int(round(new_start_x)), int(round(new_end_x)), int(round(new_start_y)), int(round(new_end_y)))
    return points


In [170]:
def implementing(size_x, size_y, points): # реализация поворота
    grid = [[0 for _ in range(size_x)] for _ in range(size_y)]
    for x, y in points:
        grid[y][x] = 1
    return grid

In [172]:
def making_situations(grid_size_x, grid_size_y,object1, object2, list, angle_list): # у object1 позиция 0,0 и он не повернут
    for i in range(-2*grid_size_x, 2*grid_size_x):
        for j in range(-2*grid_size_y, 2*grid_size_y):
            for angle in angle_list:
                list.append((i, j, angle, check_collision(object1,implementing(len(object2[0]), len(object2), turning(object2, angle)), grid_size_x, grid_size_y, i, j)))
    return list

In [ ]:
def making_collision_situations(grid_size_x, grid_size_y, max_speed_x1, max_speed_y1, max_speed_x2, max_speed_y2, object1, object2, angle_list): # object1 имеет скорость (0,0)
    x_y_dx_dy = []
    list_ = making_situations(grid_size_x, grid_size_y, object1, object2, [], angle_list)
    for x in range(-2*grid_size_x, 2*grid_size_x):
        for y in range(-2*grid_size_y, 2*grid_size_y):
            for i in range(-(max_speed_x1+max_speed_x2),max_speed_x1+max_speed_x2):
                for j in range(-(max_speed_y1+max_speed_y2),max_speed_y1+max_speed_y2):
                    nearest_angle = min(angle_list, key=lambda a: abs(a - math.atan2(j, i)))
                    if((x, y,nearest_angle, True) in list_):
                        x_y_dx_dy.append((x, y, i , j))
                    elif((x+i, y+j,nearest_angle, True) in list_):
                        x_y_dx_dy.append((x, y, i , j))
    return x_y_dx_dy

In [174]:
list_ = making_collision_situations(10, 10, 5, 5, 8, 8, ship_grid, torpedo_grid, [0, 45, 90, 135, 180, 225, 270, 315])
filename = f"{name_obj1}and{name_obj2}sets.txt"
with open(filename, "w") as file:
    for set_ in list_:
        line = " ".join(map(str, set_))
        file.write(line + "\n")

print(f"Данные успешно записаны в файл {filename}")

-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Данные успешно записаны в файл shipandtorpedosets.txt
